## Simplest Form Of Agentic Rag - Router Engine

In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Load Data

In [ ]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()





## Define LLM and Embedding model

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)




In [10]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [11]:
from llama_index.core import VectorStoreIndex, SummaryIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [12]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [15]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrievning specific context about MetaGPT paper"
    ),
)


## Define Router Query Engine

In [18]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)


In [19]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to enhance multi-agent systems based on Large Language Models (LLMs). MetaGPT improves code generation quality through role specialization, workflow management, and efficient communication mechanisms. It demonstrates superior performance in tasks like code generation and software development compared to other approaches. The document details the development process of a software application called "Drawing App" using MetaGPT, discussing the roles of agents, technical specifications, system design, implementation approach, required Python packages, file list, logic analysis, unit testing, and performance evaluation. It also explores challenges, ethics concerns, and experimental results related to MetaGPT's performance across different tasks and language models, while suggesting potential futu

In [20]:
print(len(response.source_nodes))

34


In [22]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: Agents sharing information with other agents can be summarized in a way that is useful for MetaGPT-related questions..
Agents share information with other agents through a shared message pool and a subscription mechanism. This allows them to exchange messages directly, publish structured messages, and access messages from other agents transparently. The subscription mechanism enables agents to subscribe to relevant messages based on their profiles, ensuring they obtain directional information efficiently. This structured communication interface enhances communication efficiency and prevents information overload by allowing agents to select information to follow based on their role profiles.
